In [1]:
# TẢI DỮ LIỆU VÀ CÁC PACKAGE CẦN THIẾT  

import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
import seaborn as sns 
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

file_path = "../data/imputed_data.xlsx"
xls = pd.ExcelFile(file_path)
df = xls.parse(sheet_name="Sheet1")

# display(input_df.head(5))
# display(df.shape) # có 341 hàng và 38 cột
# display(df.columns) 

In [2]:
# TẠO HAI CỘT HOMA_IR ĐỂ ĐÁNH GIÁ ĐỀ KHÁNG INSULIN

df["homa_ir"] = (df["insulin"] * df["glucose_ac"]) / 405

df["insulin_re"] = np.where(df["homa_ir"] <= 3.16, 0, 1)

In [3]:
# NHÃN BIẾN DANH ĐỊNH 

cat_vars = [
    'sex',
    'pregnancy_smoking',
    'GDM',
    'preterm_birth',
    'father_diabetes',
    'mother_diabetes',
    'education_level', 
    'CPP', 
    'insulin_re'
]

cpp_code = {0: "no", 1: "yes"}
sex_code = {0: "female", 1: "male"}
pregnancy_smoking_code = {0: "no", 1: "yes"}
# GDM_code = {"no-dia": 1, "gmd": 2, "other-dia": 3}
GDM_code = {1: "no", 2: "yes", 3: "no"}
preterm_birth_code = {0: "no", 1: "yes"}
father_diabetes_code = {0: "no", 1: "yes"}
mother_diabetes_code = {0: "no", 1: "yes"}
# education_level_code = {"senior high": 1, "vocational": 2, "graduate": 3, "college": 4}
education_level_code = {1: "low", 2: "low", 3: "high", 4: "medium"}
insulin_re_code = {0: "no", 1: "yes"}

In [4]:
# BIẾN ĐỔI NHÃN BIẾN DANH ĐỊNH

df["CPP"] = df["CPP"].map(cpp_code).astype("category")
df["sex"] = df["sex"].map(sex_code).astype("category")
df["pregnancy_smoking"] = df["pregnancy_smoking"].map(pregnancy_smoking_code).astype("category")
df["GDM"] = df["GDM"].map(GDM_code).astype("category")
df["preterm_birth"] = df["preterm_birth"].map(preterm_birth_code).astype("category")
df["father_diabetes"] = df["father_diabetes"].map(father_diabetes_code).astype("category")
df["mother_diabetes"] = df["mother_diabetes"].map(mother_diabetes_code).astype("category")
df["education_level"] = df["education_level"].map(education_level_code).astype("category")
df["insulin_re"] = df["insulin_re"].map(insulin_re_code).astype("category")

In [5]:
# TẠO MỘT SỐ CỘT MỚI 

# lbw
df["low_birth_weight"] = np.where(
    df["birth_weight_gram"] < 2500,
    "yes",
    "no"
)
df["low_birth_weight"] = df["low_birth_weight"].astype("category")

# preterm
df["preterm"] = np.where(
    df["gestational_age_week"] < 37,
    "yes",
    "no"
) # cái này đáng tin hơn, dường như cột preterm_birth khi trẻ sinh < 36 tuần
df["preterm"] = df["preterm"].astype("category")

# exclusive breast feeding time
df["breast_feeding_above_six"] = np.where(
    df["exclusive_breastfeeding_month"] >= 6,
    "yes",
    "no"
)
df["breast_feeding_above_six"] = df["breast_feeding_above_six"].astype("category")

# total breast feeding time
df["total_breastfeeding_month"] = df["exclusive_breastfeeding_month"] + df["mixed_breastfeeding_month"]
df["total_breastfeeding_above_six"] = np.where(
    df["total_breastfeeding_month"] >= 6,
    "yes",
    "no"
)
df["total_breastfeeding_above_six"] = df["total_breastfeeding_above_six"].astype("category")

# obesity or not
def determine_obesity(row):
    if pd.isna(row["zbmi"]) or pd.isna(row["age"]):
        return np.nan  # tránh lỗi khi dữ liệu thiếu

    if row["age"] < 5:
        return "yes" if row["zbmi"] > 3 else "no"
    else:
        return "yes" if row["zbmi"] > 2 else "no"
df["obesity"] = df.apply(determine_obesity, axis=1)
df["obesity"] = df["obesity"].astype("category")

# display(df["low_birth_weight"].value_counts())
# display(df["preterm"].value_counts())
# display(df["breast_feeding_above_six"].value_counts())
# display(df["total_breastfeeding_above_six"].value_counts())
# display(df["obesity"].value_counts())

In [6]:
# MỨC ĐỘ TĂNG CÂN TRONG THAI KÌ

# 1. Tạo DataFrame dữ liệu tham chiếu (GWG_ref)
GWG_ref = pd.DataFrame({
    "gestational_age_week": list(range(14, 41)),
    "p25": [-1.07, -0.45, 0.13, 0.67, 1.19, 1.69, 2.17, 2.64, 3.10, 3.54, 3.98, 4.42, 4.85, 5.28, 5.71, 6.14, 6.56, 6.99, 7.41, 7.84, 8.27, 8.70, 9.14, 9.57, 10.01, 10.45, 10.89],
    "p75": [0.65, 1.32, 1.99, 2.64, 3.29, 3.92, 4.55, 5.17, 5.79, 6.41, 7.02, 7.63, 8.24, 8.85, 9.45, 10.06, 10.67, 11.28, 11.89, 12.51, 13.12, 13.74, 14.37, 14.99, 15.62, 16.25, 16.89]
})

# 2. Tạo DataFrame dữ liệu mẫu (Có thêm dữ liệu thiếu)
target_df = df[["gestational_age_week", "gestational_weight_gain"]]

# 3. Nối (Merge) dữ liệu của bạn với bảng tham chiếu
df_merged = pd.merge(target_df, GWG_ref, on='gestational_age_week', how='left')

# 4. Phân loại mức tăng cân GWG (CÓ XỬ LÝ MISSING DATA)
# Kiểm tra dữ liệu thiếu trong 3 cột cần thiết cho việc phân loại:
# 1. Tuần tuổi thai (để merge thành công)
# 2. Tăng cân thực tế
# 3. p25/p75 (để đảm bảo tuần tuổi thai nằm trong khoảng tham chiếu)
missing_data_condition = (
    df_merged['gestational_age_week'].isna() |  # Thiếu tuần tuổi thai
    df_merged['gestational_weight_gain'].isna() | # Thiếu mức tăng cân
    df_merged['p25'].isna() # Tuần tuổi thai ngoài khoảng 14-40
)

conditions = [
    missing_data_condition,                                       # Điều kiện 1: Dữ liệu bị thiếu
    (df_merged['gestational_weight_gain'] < df_merged['p25']),     # Điều kiện 2: Dưới p25
    (df_merged['gestational_weight_gain'] > df_merged['p75']),     # Điều kiện 3: Trên p75
]

choices = [
    "Không phân loại", # Kết quả cho Điều kiện 1
    "Không đủ",                               # Kết quả cho Điều kiện 2
    "Quá nhiều",                              # Kết quả cho Điều kiện 3
]

# Áp dụng các điều kiện. 'Phù hợp' là mặc định (p25 <= GWG <= p75)
df['gwg_category'] = np.select(conditions, choices, default='Phù hợp')
df['gwg_category'] = df['gwg_category'].replace("Không phân loại", np.nan)
df['gwg_category'] = df['gwg_category'].astype("category")

# 5. In kết quả
# display(df['gwg_category'].value_counts())


In [7]:
# THỐNG KÊ CÁC LOẠI BIẾN

numeric_vars = df.select_dtypes(["int", "float"]).columns.to_list()
# display(len(numeric_vars)) # 32 biến số số học 

cat_vars = df.select_dtypes("category").columns.to_list()
# display(len(cat_vars)) # 15 biến số danh định

In [8]:
# THỐNG KÊ MÔ TẢ CHO CÁC BIẾN SỐ ĐỊNH TÍNH

for x in cat_vars:
    column = df[x]
    n_count = column.value_counts()
    percentages = (column.value_counts(normalize=True) * 100).round(1)
    report = pd.DataFrame({
        x: n_count.index, 
        "count": n_count.values, 
        "percentages": percentages.values
    })
    # print(f"{x}: {column.notna().sum()}/341 non-missing values\n")
    # display(report)

In [9]:
# KIỂM TRA PHÂN PHỐI CHUẨN 

from scipy.stats import shapiro

def normality_test(column_name):
    stat, p = shapiro(df[column_name].dropna())  # tránh lỗi nếu có NaN - PHẢI CÓ DROPNA
    return p < 0.05  # True nếu KHÔNG chuẩn

non_normality_columns = [col for col in numeric_vars if normality_test(col)]
normality_columns = [col for col in numeric_vars if not normality_test(col)]

# print(non_normality_columns) 
# print(len(non_normality_columns))
# print(normality_columns)
# print(len(normality_columns))

In [10]:
# VẼ BIỂU ĐỒ KIỂM ĐỊNH PHÂN PHỐI CHUẨN

# variables = ['calories(kcal)', 'total_carbohydrates(g)']

# for var in variables:

#     plt.figure(figsize=(4, 3))  # 👈 điều chỉnh kích thước tại đây
#     plt.hist(df[var], bins=50, color='steelblue', edgecolor='black')
#     plt.title(var)
#     plt.xlabel(var)
#     plt.ylabel("Frequency")
#     plt.tight_layout()
#     plt.show()


In [ ]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Giả sử df đang ở dạng wide:
# # df = pd.DataFrame({'var1': ..., 'var2': ..., 'var3': ...})

# # 1️⃣ Chuyển sang dạng long (giống pivot_longer trong R)
# df_long = df.melt(value_vars=['var1', 'var2', 'var3'], 
#                   var_name='variable', value_name='value')

# # 2️⃣ Vẽ histogram với facet theo biến
# sns.displot(
#     data=df_long,
#     x='value',
#     col='variable',         # tách biểu đồ theo biến
#     col_wrap=3,             # số cột mỗi hàng (tuỳ chỉnh)
#     kde=False,              # vẽ histogram (True sẽ thêm density curve)
#     color='skyblue'
# )

# plt.suptitle("Histogram các biến liên tục", y=1.02)
# plt.show()


In [11]:
# THỐNG KÊ MÔ TẢ CHO CÁC BIẾN SỐ ĐỊNH LƯỢNG KHÔNG CHUẨN 

# df[non_normality_columns].describe().round(1).loc[['count', 'min', '25%', '50%', '75%', 'max']]


In [12]:
# THỐNG KÊ MÔ TẢ CHO CÁC BIẾN SỐ ĐỊNH LƯỢNG PHÂN PHỐI CHUẨN

# df[normality_columns].describe().round(1).loc[['count', 'min', 'max', 'mean', 'std']]

In [ ]:
# NUMERIC EDA DÙNG GROUPBY THEO TÌNH TRẠNG ĐỀ KHÁNG INSULIN

num_tables = []

for var in numeric_vars:  # danh sách các biến số liên tục
    stats = (
        df.groupby("insulin_re", observed=True)[var]
        .agg(["mean", "std"])
        .round(1)
    )
    
    # Gộp mean và sd thành chuỗi "mean (sd)"
    summary = stats["mean"].astype(str) + " (" + stats["std"].astype(str) + ")"
    summary = summary.to_frame().T  # chuyển sang dạng hàng ngang # phải có to_frame() vì đang là series
    summary.insert(0, "var", var)
    num_tables.append(summary)

num_report = pd.concat(num_tables, ignore_index=True)

num_report.head(3)


insulin_re,var,no,yes
0,age,10.2 (2.0),10.6 (1.8)
1,zbmi,0.2 (1.5),1.5 (1.4)
2,insulin,8.4 (3.4),25.5 (17.3)


In [16]:
# NUMERIC EDA/INFER THEO TÌNH TRẠNG ĐỀ KHÁNG INSULIN

from scipy import stats

results = []

for var in numeric_vars:
    group = df[var]
    group_yes = df.loc[df["insulin_re"] == "yes", var]
    group_no  = df.loc[df["insulin_re"] == "no", var]

    # Kiểm định t-test độc lập # không chuẩn - nên dùng mannwhitneyu
    t_stat, p_val = stats.ttest_ind(group_yes, group_no, equal_var=False)  # Welch's t-test # thực tế phần lớn không phân phối chuẩn

    results.append({
        "var": var,
        "mean_sd":  f"{group.mean().round(1)} ({group.std().round(1)})",
        "mean_sd_yes": f"{group_yes.mean().round(1)} ({group_yes.std().round(1)})",
        "mean_sd_no": f"{group_no.mean().round(1)} ({group_no.std().round(1)})",
        "p_value": round(p_val, 3)
    })

insulin_re_ttest_report = pd.DataFrame(results)

insulin_re_ttest_report.head(10)


,var,mean_sd,mean_sd_yes,mean_sd_no,p_value
0,age,10.3 (2.0),10.6 (1.8),10.2 (2.0),0.043
1,zbmi,0.6 (1.6),1.5 (1.4),0.2 (1.5),0.000
2,insulin,14.1 (13.1),25.5 (17.3),8.4 (3.4),0.000
3,glucose_ac,89.4 (16.5),91.2 (9.1),88.5 (19.1),0.076
4,cholesterol,166.4 (30.0),166.1 (30.5),166.6 (29.8),0.900
5,TG,78.1 (42.3),96.2 (45.5),69.0 (37.5),0.000
6,HDL,61.6 (13.8),56.7 (13.1),64.1 (13.5),0.000
7,cortisol,8.8 (5.9),9.5 (7.7),8.5 (4.7),0.244
8,waist,66.4 (12.2),73.0 (13.0),63.1 (10.4),0.000
9,hip,77.7 (10.4),83.0 (10.8),75.1 (9.1),0.000


In [17]:
# NUMERIC EDA/INFER THEO GIỚI VÀ TÌNH TRẠNG ĐỀ KHÁNG INSULIN


from scipy import stats


male_results = []

for var in numeric_vars:
    group = df.loc[df["sex"] == "male", var]
    group_yes = df.loc[(df["sex"] == "male") & (df["insulin_re"] == "yes"), var]
    group_no  = df.loc[(df["sex"] == "male") & (df["insulin_re"] == "no"), var]

    # Kiểm định t-test độc lập # không chuẩn - nên dùng mannwhitneyu
    t_stat, p_val = stats.ttest_ind(group_yes, group_no, equal_var=False)  # Welch's t-test # thực tế phần lớn không phân phối chuẩn

    male_results.append({
        "male_sex": "male",
        "var": var,
        "male_mean_sd":  f"{group.mean().round(1)} ({group.std().round(1)})",
        "male_mean_sd_yes": f"{group_yes.mean().round(1)} ({group_yes.std().round(1)})",
        "male_mean_sd_no": f"{group_no.mean().round(1)} ({group_no.std().round(1)})",
        "male_p_value": round(p_val, 3)
    })

male_ttest_report = pd.DataFrame(male_results)

female_results = []

for var in numeric_vars:
    group = df.loc[df["sex"] == "female", var]
    group_yes = df.loc[(df["sex"] == "female") & (df["insulin_re"] == "yes"), var]
    group_no  = df.loc[(df["sex"] == "female") & (df["insulin_re"] == "no"), var]

    # Kiểm định t-test độc lập # không chuẩn - nên dùng mannwhitneyu
    t_stat, p_val = stats.ttest_ind(group_yes, group_no, equal_var=False)  # Welch's t-test # thực tế phần lớn không phân phối chuẩn

    female_results.append({
        "female_sex": "female",
        "var": var,
        "female_mean_sd":  f"{group.mean().round(1)} ({group.std().round(1)})",
        "female_mean_sd_yes": f"{group_yes.mean().round(1)} ({group_yes.std().round(1)})",
        "female_mean_sd_no": f"{group_no.mean().round(1)} ({group_no.std().round(1)})",
        "female_p_value": round(p_val, 3)
    })

female_ttest_report = pd.DataFrame(female_results)

sex_insulin_re_ttest_report = pd.merge(male_ttest_report, female_ttest_report, how="outer", on="var")

sex_insulin_re_ttest_report.head(10)


,male_sex,var,male_mean_sd,male_mean_sd_yes,male_mean_sd_no,male_p_value,female_sex,female_mean_sd,female_mean_sd_yes,female_mean_sd_no,female_p_value
0,male,DBP,57.5 (15.7),54.8 (19.2),58.9 (13.3),0.210,female,58.4 (15.5),58.4 (16.3),58.5 (15.3),0.974
1,male,HDL,62.1 (14.2),57.2 (13.1),64.7 (14.1),0.004,female,61.4 (13.6),56.3 (13.2),63.8 (13.2),0.000
2,male,SBP,88.8 (22.4),86.0 (27.3),90.2 (19.3),0.370,female,86.3 (20.8),87.6 (22.2),85.7 (20.1),0.535
3,male,TG,76.0 (48.8),91.8 (43.1),67.6 (49.7),0.005,female,79.3 (38.0),99.0 (47.1),69.8 (28.3),0.000
4,male,age,11.4 (1.9),11.5 (1.5),11.3 (2.0),0.633,female,9.7 (1.7),10.1 (1.8),9.5 (1.7),0.030
5,male,birth_weight_gram,3035.4 (551.6),3022.7 (592.6),3042.2 (532.2),0.856,female,2980.1 (496.4),2942.9 (540.3),2998.3 (474.4),0.466
6,male,calories(kcal),1778.1 (426.6),1760.8 (469.3),1787.3 (404.8),0.751,female,1541.0 (415.4),1557.7 (429.1),1532.8 (409.8),0.687
7,male,cholesterol,162.2 (32.6),164.0 (31.3),161.3 (33.5),0.651,female,169.0 (28.1),167.5 (30.2),169.7 (27.1),0.616
8,male,cortisol,9.4 (7.3),10.4 (10.6),8.9 (4.8),0.373,female,8.5 (4.9),8.9 (5.2),8.4 (4.7),0.467
9,male,exclusive_breastfeeding_month,3.3 (2.6),3.2 (2.7),3.3 (2.6),0.755,female,3.4 (2.5),3.2 (2.5),3.5 (2.5),0.495


In [ ]:
# CATEGORICAL EDA/INFER THEO TÌNH TRẠNG ĐỀ KHÁNG INSULIN

from scipy import stats

cat_tables = []

for var in cat_vars: 
    # Bảng tần số
    count = pd.crosstab(df[var], df["insulin_re"])
    percentages = (pd.crosstab(df[var], df["insulin_re"], normalize="columns") * 100).round(1)

    # Bảng tóm tắt: count + percentage
    summary = count.astype(str) + " (" + percentages.astype(str) + "%)"
    summary = summary.reset_index(drop=False)
    summary.rename(columns={var: "labels"}, inplace=True)
    summary.insert(0, "var", var)

    # --- Kiểm định ---
    if count.shape == (2, 2):  # Fisher’s exact test
        _, p = stats.fisher_exact(count)
        test_name = "Fisher"
    else:  # Chi-square test
        _, p, _, _ = stats.chi2_contingency(count)
        test_name = "Chi-square"

    # Thêm thông tin kiểm định vào bảng
    summary["test"] = test_name
    summary["p_value"] = p.round(3)

    cat_tables.append(summary)

# Gộp tất cả biến lại
cat_report = pd.concat(cat_tables, ignore_index=True)

# Hiển thị
cat_report.head(10)


insulin_re,var,labels,no,yes,test,p_value
0,CPP,no,100 (44.1%),36 (31.6%),Fisher,0.035
1,CPP,yes,127 (55.9%),78 (68.4%),Fisher,0.035
2,sex,female,144 (63.4%),70 (61.4%),Fisher,0.723
3,sex,male,83 (36.6%),44 (38.6%),Fisher,0.723
4,pregnancy_smoking,no,225 (99.1%),114 (100.0%),Fisher,0.554
5,pregnancy_smoking,yes,2 (0.9%),0 (0.0%),Fisher,0.554
6,GDM,no,225 (99.1%),106 (93.0%),Fisher,0.003
7,GDM,yes,2 (0.9%),8 (7.0%),Fisher,0.003
8,preterm_birth,no,217 (95.6%),103 (90.4%),Fisher,0.092
9,preterm_birth,yes,10 (4.4%),11 (9.6%),Fisher,0.092


In [ ]:
# CATEGORICAL EDA/INFER THEO GIỚI VÀ TÌNH TRẠNG ĐỀ KHÁNG INSULIN

import pandas as pd
from scipy import stats

cat_tables = []

for var in cat_vars:
    # Bảng đếm & phần trăm
    count = pd.crosstab(df[var], [df["sex"], df["insulin_re"]])
    percentages = (pd.crosstab(df[var], [df["sex"], df["insulin_re"]], normalize="columns") * 100).round(1)

    # Ghép số lượng và %
    summary = count.astype(str) + " (" + percentages.astype(str) + "%)"
    summary = summary.reset_index(drop=False)
    summary.rename(columns={var: "labels"}, inplace=True)
    summary.insert(0, "var", var)

    # --- KIỂM ĐỊNH TRONG TỪNG NHÓM GIỚI ---
    p_values = {}
    for sex in df["sex"].unique():
        sub_df = df[df["sex"] == sex]
        tab = pd.crosstab(sub_df[var], sub_df["insulin_re"])

        if tab.shape == (2, 2):
            _, p = stats.fisher_exact(tab)
            test = "Fisher"
        else:
            _, p, _, _ = stats.chi2_contingency(tab)
            test = "Chi-square"

        p_values[f"{sex}_p"] = round(p, 3)
        p_values[f"{sex}_test"] = test

    # Gắn kết quả kiểm định
    for key, val in p_values.items():
        summary[key] = val

    cat_tables.append(summary)

# Gộp tất cả lại
cat_report = pd.concat(cat_tables, ignore_index=True)

# Hiển thị kết quả
display(cat_report.head(10))


sex                       var  labels        female                      male  \
insulin_re                                       no          yes           no   
0                         CPP      no    53 (36.8%)   23 (32.9%)   47 (56.6%)   
1                         CPP     yes    91 (63.2%)   47 (67.1%)   36 (43.4%)   
2                         sex  female  144 (100.0%)  70 (100.0%)     0 (0.0%)   
3                         sex    male      0 (0.0%)     0 (0.0%)  83 (100.0%)   
4           pregnancy_smoking      no  144 (100.0%)  70 (100.0%)   81 (97.6%)   
5           pregnancy_smoking     yes      0 (0.0%)     0 (0.0%)     2 (2.4%)   
6                         GDM      no   142 (98.6%)   65 (92.9%)  83 (100.0%)   
7                         GDM     yes      2 (1.4%)     5 (7.1%)     0 (0.0%)   
8               preterm_birth      no   138 (95.8%)   64 (91.4%)   79 (95.2%)   
9               preterm_birth     yes      6 (4.2%)     6 (8.6%)     4 (4.8%)   

sex                     male_p   male_test female_p female_test  
insulin_re          yes                                          
0            13 (29.5%)  0.005      Fisher    0.649      Fisher  
1            31 (70.5%)  0.005      Fisher    0.649      Fisher  
2              0 (0.0%)  1.000  Chi-square    1.000  Chi-square  
3           44 (100.0%)  1.000  Chi-square    1.000  Chi-square  
4           44 (100.0%)  0.544      Fisher    1.000  Chi-square  
5              0 (0.0%)  0.544      Fisher    1.000  Chi-square  
6            41 (93.2%)  0.040      Fisher    0.039      Fisher  
7              3 (6.8%)  0.040      Fisher    0.039      Fisher  
8            39 (88.6%)  0.274      Fisher    0.213      Fisher  
9             5 (11.4%)  0.274      Fisher    0.213      Fisher